In [1]:
#!pip3 install numpy
#!pip3 install pandas
#!pip3 install requests

import numpy as np
import re
import requests
import pandas as pd
import seaborn as sns
from functools import reduce

# Igel
from igel import Igel

# Compare Algorithms
import matplotlib.pyplot as plt
from matplotlib import rcParams

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def clean_dataset(df):
    #assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    #df.dropna(inplace=True)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df
    #return df[indices_to_keep].astype(np.float64)
    

def clean_dataset_int(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df = df.replace(r'\D+', '', regex=True)
    return df[indices_to_keep].astype(np.float64)

In [3]:
priority_scorer = {
    '10' : 'Critical',
    '9' : 'Critical',
    '8' : 'Critical',
    '7' : 'High',
    '6' : 'High',
    '5' : 'Medium',
    '4' : 'Medium',
    '3' : 'Low',
    '2' : 'Low',
    '1.0' : 'Low',
    '0.0' : 'Low',
}

priority_mapping = {
    "Critical" : 10,
    "High" : 7.5,
    "Medium" : 5,
    "Low" : 2.5,
    "Unknown" : 0,
}

# What we consider to be highly important categories of information
highCategoriser = {
    0.0 : 'Other-Advice',
    1.0 : 'Other-Advice',
    2.0 : 'Report-CleanUp',
    3.0 : 'ContextualInformation',
    4.0 : 'Other-ContextualInformation',
    5.0 : 'CallToAction-Donations',
    6.0 : 'Report-EmergingThreats',
    7.0 : 'Report-Factoid',
    8.0 : 'Report-FirstPartyObservation',
    9.0 : 'Request-GoodsServices',
    10.0 : 'Report-Hashtags',
    11.0 : 'Request-InformationWanted',
    12.0 : 'Other-Irrelevant',
    13.0 : 'Report-Location',
    14.0 : 'CallToAction-MovePeople',
    15.0 : 'Report-MultimediaShare',
    16.0 : 'Report-NewSubEvent',
    17.0 : 'Report-News',
    18.0 : 'Report-Official',
    19.0 : 'Report-OriginalEvent',
    20.0 : 'Request-SearchAndRescue',
    21.0 : 'Other-Sentiment',
    22.0 : 'Report-ServiceAvailable',
    23.0 : 'Report-ThirdPartyObservation',
    24.0 : 'CallToAction-Volunteer',
    25.0 : 'Report-Weather',
    26.0 : 'hmm',
}

event_int_map =	{
  "guatemalaEarthquake2012": 7,
  "joplinTornado2011": 16,
  "athensEarthquake2020": 35,
  "baltimoreFlashFlood2020": 36,
  "brooklynBlockPartyShooting2020": 37,
  "daytonOhioShooting2020": 38,
  "elPasoWalmartShooting2020": 39,
  "gilroygarlicShooting2020": 40,
  "hurricaneBarry2020": 41,
  "indonesiaEarthquake2020": 42,
  "keralaFloods2020": 43,
  "myanmarFloods2020": 44,
  "papuaNewguineaEarthquake2020": 45,
  "siberianWildfires2020": 46,
  "typhoonKrosa2020": 47,
  "typhoonLekima2020": 48,
  "whaleyBridgeCollapse2020": 49
}

mymap = {'Advice':1, 'CleanUp':2, 'ContextualInformation':3, 'Discussion':4, 'Donations':5, 
        'EmergingThreats':6, 'Factoid':7, 'FirstPartyObservation':8, 'GoodsServices':9, 'Hashtags':10, 
        'InformationWanted':11,'Irrelevant':12, 'Location':13, 'MovePeople':14, 
         'MultimediaShare':15, 'NewSubEvent':16, 'News':17,
        'Official':18, 'OriginalEvent':19, 'SearchAndRescue':20, 'Sentiment':21, 'ServiceAvailable':22, 
         'ThirdPartyObservation':23,'Volunteer':24, 'Weather':25}

## Feature Vector

Load the feature vector in from Play

In [4]:
#url = 'http://localhost:9000/stored_tweets'
url = 'http://tweetminer-2336003gproject.ida.dcs.gla.ac.uk/stored_tweets'

data = requests.get(url).json()

In [5]:
print(data)

{'fileName': 'data/test-tweets/test/200.jsonl', 'tweets': [{'tweet_id': 1157452613228433410, 'tweet_text': '[earthquake, mag, depth, utc, location, bmkg, aeic]', 'features': {'numb_of_urls': None, 'numb_of_hashtags': 0.0, 'numb_of_personal_pronouns': 0.0, 'numb_of_present_tenses': 0.0, 'weighted_length': 181.0, 'permillage': 646.0, 'tweet_created_at': 1564793182000.0, 'tweet_id_str': 1.1574526132284334e+18, 'positive_sentiment': 0.0, 'negative_sentiment': 0.0, 'numb_of_mentions': 0.0, 'numb_of_media': 1.0, 'numb_of_past_tenses': 0.0, 'numb_of_weird_chars': 14.0, 'numb_of_questions': 0.0, 'numb_of_emoticons': 0.0, 'numb_of_swearing_word': 0.0, 'numb_of_slang_words': 0.0, 'numb_of_intensifiers': 0.0, 'tweet_length': 51.0, 'userFollowersCount': 21681.0, 'userFriendsCount': 710.0, 'user_numb_of_tweets': 0.0, 'user_list_count': 111.0, 'dict_precision': 0.0, 'dict_recall': 0.0, 'dict_f_measure': 0.0, 'offset': 0.0, 'is_verified': 0.0}, 'tfIdf': {'mag': 0.6578815, 'aeic': 0.6578815, 'depth': 

In [6]:
df = pd.DataFrame(data)

In [7]:
df = pd.json_normalize(df['tweets'])

In [8]:
#df['sentiment']

In [9]:
df['sentiment'] = df[df['sentiment'].apply(lambda x:pd.Series(x))]
df_sentiment=pd.concat([df['sentiment'].str.split(',', expand=True)], axis=1, keys="s")
df_sentiment

INFO - NumExpr defaulting to 8 threads.


,s
,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
195,NaN
196,NaN
197,NaN


In [10]:
df.join(df_sentiment)

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.mag,tfIdf.aeic,tfIdf.depth,tfIdf.utc,tfIdf.bmkg,tfIdf.location,tfIdf.earthquake,tfIdf.island,tfIdf.strikes,tfIdf.magnitude,tfIdf.sumatra,tfIdf.indonesia,tfIdf.news,tfIdf.including,tfIdf.main,tfIdf.centred,tfIdf.bali,tfIdf.java,tfIdf.powerful,tfIdf.islands,tfIdf.rocked,tfIdf.quake,tfIdf.safe,tfIdf.feel,tfIdf.jakarta,tfIdf.stay,tfIdf.app,tfIdf.southwest,tfIdf.amp,tfIdf.minutes,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.aug,tfIdf.fri,tfIdf.info,tfIdf.postgresql,tfIdf.united,tfIdf.alert,tfIdf.rate,tfIdf.scrum,tfIdf.developer,tfIdf.kingdom,tfIdf.anxiety,tfIdf.tsunami,tfIdf.bring,tfIdf.sign,tfIdf.rocks,tfIdf.warning,tfIdf.issued,tfIdf.sunda,tfIdf.lives,tfIdf.selat,tfIdf.banten,tfIdf.potential,tfIdf.sr,tfIdf.country,tfIdf.cities,tfIdf.live,tfIdf.hits,tfIdf.indonesias,tfIdf.dead,tfIdf.panic,tfIdf.indonesian,tfIdf.kills,tfIdf.strong,tfIdf.displaced,tfIdf.dwellers,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.struck,tfIdf.coastal,tfIdf.fear,tfIdf.super,tfIdf.drunk,tfIdf.spinning,tfIdf.wild,tfIdf.fears,tfIdf.ground,tfIdf.triggers,tfIdf.urged,tfIdf.capital,tfIdf.swaying,tfIdf.died,tfIdf.heart,tfIdf.authorities,tfIdf.hit,tfIdf.attack,tfIdf.buildings,tfIdf.damaged,tfIdf.houses,tfIdf.urging,tfIdf.rocking,tfIdf.officials,tfIdf.initially,tfIdf.coast,tfIdf.cancelled,tfIdf.populated,tfIdf.heavily,tfIdf.injured,tfIdf.worldnews,tfIdf.usgs,tfIdf.teacup,tfIdf.poodle,tfIdf.tiny,tfIdf.jual,tfIdf.black,tfIdf.betina,tfIdf.quality,tfIdf.toy,tfIdf.cbs,tfIdf.lifted,tfIdf.software,tfIdf.subscribe,tfIdf.learn,tfIdf.link,tfIdf.channel,tfIdf.pay,tfIdf.apwebier,tfIdf.click,tfIdf.languege,tfIdf.field,tfIdf.money,tfIdf.watch,tfIdf.success,tfIdf.earn,tfIdf.free,tfIdf.beautiful,tfIdf.destination,tfIdf.pray,tfIdf.assisting,tfIdf.chapter,tfIdf.code,tfIdf.languages,tfIdf.bright,tfIdf.students,tfIdf.competition,tfIdf.special,tfIdf.coding,tfIdf.victor,tfIdf.school,tfIdf.future,tfIdf.atl,tfIdf.montgomery,tfIdf.farm,tfIdf.bdpa,tfIdf.laptops,tfIdf.programming,tfIdf.scary,tfIdf.bed,tfIdf.south,tfIdf.moves,tfIdf.literally,tfIdf.breathe,tfIdf.damage,tfIdf.reports,tfIdf.prompting,tfIdf.tremors,tfIdf.office,tfIdf.casualties,tfIdf.prayforjakarta,tfIdf.west,tfIdf.breaking,tfIdf.huge,tfIdf.rattled,tfIdf.residents,tfIdf.thankyou,tfIdf.inform,tfIdf.fine,tfIdf.release,tfIdf.loves,tfIdf.update,tfIdf.source,tfIdf.naturaldisasters,tfIdf.strongest,tfIdf.ve,tfIdf.yup,tfIdf.brush,tfIdf.ease,tfIdf.basics,tfIdf.august,tfIdf.expan,tfIdf.skills,tfIdf.write,tfIdf.updated,tfIdf.guide,tfIdf.apps,tfIdf.pretty,tfIdf.university,tfIdf.searching,tfIdf.reading,tfIdf.american,tfIdf.marrigge,tfIdf.girl,tfIdf.boy,tfIdf.arabian,tfIdf.books,tfIdf.pdf,tfIdf.teacher,tfIdf.budg,tfIdf.cambridge,tfIdf.diploma,tfIdf.internet,tfIdf.yall,tfIdf.boiz,tfIdf.ripcurrents,tfIdf.scared,tfIdf.gempa,tfIdf.mins,tfIdf.report,tfIdf.eqalerts,tfIdf.hello,tfIdf.isya,tfIdf.agency,tfIdf.geophysics,tfIdf.saturday,tfIdf.feared,tfIdf.disaster,tfIdf.guys,tfIdf.lifts,tfIdf.shallow,tfIdf.image,tfIdf.sw,tfIdf.epicenter,tfIdf.southern,tfIdf.southernsumatra,tfIdf.occurred,tfIdf.streets,tfIdf.fleeing,tfIdf.friday,tfIdf.evening,tfIdf.province,tfIdf.generating,tfIdf.indonesiaearthquake,tfIdf.risked,tfIdf.monitors,tfIdf.fecha,tfIdf.t

In [11]:
df['embeddings'] 

0      {1.3965459,4.2288527,1.07686,3.4551997,2.08345...
1      {3.9799151,-2.65434,0.3603499,4.93875,1.886745...
2      {16.603096,-0.8937713,-3.6050925,19.039347,9.1...
3      {2.398402,-0.056690022,0.23064996,1.01621,3.55...
4      {2.1278234,6.669227,2.9769623,11.665141,4.8706...
                             ...                        
195    {3.164166,-0.5398301,0.87061083,3.9886048,3.60...
196    {-0.44311205,2.32839,0.42229247,2.3441799,1.52...
197    {12.958414,-0.6023939,-2.180224,21.534632,11.2...
198    {10.661394,-0.9631202,-1.0258089,10.615733,10....
199    {3.127284,-1.16586,-0.5944201,4.6820498,1.7022...
Name: embeddings, Length: 200, dtype: object

In [12]:
df['embeddings'] = df['embeddings'].str.extract('(\d+)', expand=False)
df['embeddings'] = df['embeddings'].str[1:]
df['embeddings']

0       
1       
2      6
3       
4       
      ..
195     
196     
197    2
198    0
199     
Name: embeddings, Length: 200, dtype: object

In [13]:
df_embeddings = pd.concat([df['embeddings'].str.split(',', expand=True)], axis=1, keys="e")
df_embeddings

,e
,0
0,
1,
2,6
3,
4,
...,...
195,
196,
197,2


In [14]:
df.join(df_embeddings)

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.mag,tfIdf.aeic,tfIdf.depth,tfIdf.utc,tfIdf.bmkg,tfIdf.location,tfIdf.earthquake,tfIdf.island,tfIdf.strikes,tfIdf.magnitude,tfIdf.sumatra,tfIdf.indonesia,tfIdf.news,tfIdf.including,tfIdf.main,tfIdf.centred,tfIdf.bali,tfIdf.java,tfIdf.powerful,tfIdf.islands,tfIdf.rocked,tfIdf.quake,tfIdf.safe,tfIdf.feel,tfIdf.jakarta,tfIdf.stay,tfIdf.app,tfIdf.southwest,tfIdf.amp,tfIdf.minutes,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.aug,tfIdf.fri,tfIdf.info,tfIdf.postgresql,tfIdf.united,tfIdf.alert,tfIdf.rate,tfIdf.scrum,tfIdf.developer,tfIdf.kingdom,tfIdf.anxiety,tfIdf.tsunami,tfIdf.bring,tfIdf.sign,tfIdf.rocks,tfIdf.warning,tfIdf.issued,tfIdf.sunda,tfIdf.lives,tfIdf.selat,tfIdf.banten,tfIdf.potential,tfIdf.sr,tfIdf.country,tfIdf.cities,tfIdf.live,tfIdf.hits,tfIdf.indonesias,tfIdf.dead,tfIdf.panic,tfIdf.indonesian,tfIdf.kills,tfIdf.strong,tfIdf.displaced,tfIdf.dwellers,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.struck,tfIdf.coastal,tfIdf.fear,tfIdf.super,tfIdf.drunk,tfIdf.spinning,tfIdf.wild,tfIdf.fears,tfIdf.ground,tfIdf.triggers,tfIdf.urged,tfIdf.capital,tfIdf.swaying,tfIdf.died,tfIdf.heart,tfIdf.authorities,tfIdf.hit,tfIdf.attack,tfIdf.buildings,tfIdf.damaged,tfIdf.houses,tfIdf.urging,tfIdf.rocking,tfIdf.officials,tfIdf.initially,tfIdf.coast,tfIdf.cancelled,tfIdf.populated,tfIdf.heavily,tfIdf.injured,tfIdf.worldnews,tfIdf.usgs,tfIdf.teacup,tfIdf.poodle,tfIdf.tiny,tfIdf.jual,tfIdf.black,tfIdf.betina,tfIdf.quality,tfIdf.toy,tfIdf.cbs,tfIdf.lifted,tfIdf.software,tfIdf.subscribe,tfIdf.learn,tfIdf.link,tfIdf.channel,tfIdf.pay,tfIdf.apwebier,tfIdf.click,tfIdf.languege,tfIdf.field,tfIdf.money,tfIdf.watch,tfIdf.success,tfIdf.earn,tfIdf.free,tfIdf.beautiful,tfIdf.destination,tfIdf.pray,tfIdf.assisting,tfIdf.chapter,tfIdf.code,tfIdf.languages,tfIdf.bright,tfIdf.students,tfIdf.competition,tfIdf.special,tfIdf.coding,tfIdf.victor,tfIdf.school,tfIdf.future,tfIdf.atl,tfIdf.montgomery,tfIdf.farm,tfIdf.bdpa,tfIdf.laptops,tfIdf.programming,tfIdf.scary,tfIdf.bed,tfIdf.south,tfIdf.moves,tfIdf.literally,tfIdf.breathe,tfIdf.damage,tfIdf.reports,tfIdf.prompting,tfIdf.tremors,tfIdf.office,tfIdf.casualties,tfIdf.prayforjakarta,tfIdf.west,tfIdf.breaking,tfIdf.huge,tfIdf.rattled,tfIdf.residents,tfIdf.thankyou,tfIdf.inform,tfIdf.fine,tfIdf.release,tfIdf.loves,tfIdf.update,tfIdf.source,tfIdf.naturaldisasters,tfIdf.strongest,tfIdf.ve,tfIdf.yup,tfIdf.brush,tfIdf.ease,tfIdf.basics,tfIdf.august,tfIdf.expan,tfIdf.skills,tfIdf.write,tfIdf.updated,tfIdf.guide,tfIdf.apps,tfIdf.pretty,tfIdf.university,tfIdf.searching,tfIdf.reading,tfIdf.american,tfIdf.marrigge,tfIdf.girl,tfIdf.boy,tfIdf.arabian,tfIdf.books,tfIdf.pdf,tfIdf.teacher,tfIdf.budg,tfIdf.cambridge,tfIdf.diploma,tfIdf.internet,tfIdf.yall,tfIdf.boiz,tfIdf.ripcurrents,tfIdf.scared,tfIdf.gempa,tfIdf.mins,tfIdf.report,tfIdf.eqalerts,tfIdf.hello,tfIdf.isya,tfIdf.agency,tfIdf.geophysics,tfIdf.saturday,tfIdf.feared,tfIdf.disaster,tfIdf.guys,tfIdf.lifts,tfIdf.shallow,tfIdf.image,tfIdf.sw,tfIdf.epicenter,tfIdf.southern,tfIdf.southernsumatra,tfIdf.occurred,tfIdf.streets,tfIdf.fleeing,tfIdf.friday,tfIdf.evening,tfIdf.province,tfIdf.generating,tfIdf.indonesiaearthquake,tfIdf.risked,tfIdf.monitors,tfIdf.fecha,tfIdf.t

In [15]:
df_emb_sent=df_sentiment.join(df_embeddings)
df_emb_sent

,s,e
,0,0
0,NaN,
1,NaN,
2,NaN,6
3,NaN,
4,NaN,
...,...,...
195,NaN,
196,NaN,
197,NaN,2


In [16]:
df.join(df_emb_sent)

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.mag,tfIdf.aeic,tfIdf.depth,tfIdf.utc,tfIdf.bmkg,tfIdf.location,tfIdf.earthquake,tfIdf.island,tfIdf.strikes,tfIdf.magnitude,tfIdf.sumatra,tfIdf.indonesia,tfIdf.news,tfIdf.including,tfIdf.main,tfIdf.centred,tfIdf.bali,tfIdf.java,tfIdf.powerful,tfIdf.islands,tfIdf.rocked,tfIdf.quake,tfIdf.safe,tfIdf.feel,tfIdf.jakarta,tfIdf.stay,tfIdf.app,tfIdf.southwest,tfIdf.amp,tfIdf.minutes,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.aug,tfIdf.fri,tfIdf.info,tfIdf.postgresql,tfIdf.united,tfIdf.alert,tfIdf.rate,tfIdf.scrum,tfIdf.developer,tfIdf.kingdom,tfIdf.anxiety,tfIdf.tsunami,tfIdf.bring,tfIdf.sign,tfIdf.rocks,tfIdf.warning,tfIdf.issued,tfIdf.sunda,tfIdf.lives,tfIdf.selat,tfIdf.banten,tfIdf.potential,tfIdf.sr,tfIdf.country,tfIdf.cities,tfIdf.live,tfIdf.hits,tfIdf.indonesias,tfIdf.dead,tfIdf.panic,tfIdf.indonesian,tfIdf.kills,tfIdf.strong,tfIdf.displaced,tfIdf.dwellers,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.struck,tfIdf.coastal,tfIdf.fear,tfIdf.super,tfIdf.drunk,tfIdf.spinning,tfIdf.wild,tfIdf.fears,tfIdf.ground,tfIdf.triggers,tfIdf.urged,tfIdf.capital,tfIdf.swaying,tfIdf.died,tfIdf.heart,tfIdf.authorities,tfIdf.hit,tfIdf.attack,tfIdf.buildings,tfIdf.damaged,tfIdf.houses,tfIdf.urging,tfIdf.rocking,tfIdf.officials,tfIdf.initially,tfIdf.coast,tfIdf.cancelled,tfIdf.populated,tfIdf.heavily,tfIdf.injured,tfIdf.worldnews,tfIdf.usgs,tfIdf.teacup,tfIdf.poodle,tfIdf.tiny,tfIdf.jual,tfIdf.black,tfIdf.betina,tfIdf.quality,tfIdf.toy,tfIdf.cbs,tfIdf.lifted,tfIdf.software,tfIdf.subscribe,tfIdf.learn,tfIdf.link,tfIdf.channel,tfIdf.pay,tfIdf.apwebier,tfIdf.click,tfIdf.languege,tfIdf.field,tfIdf.money,tfIdf.watch,tfIdf.success,tfIdf.earn,tfIdf.free,tfIdf.beautiful,tfIdf.destination,tfIdf.pray,tfIdf.assisting,tfIdf.chapter,tfIdf.code,tfIdf.languages,tfIdf.bright,tfIdf.students,tfIdf.competition,tfIdf.special,tfIdf.coding,tfIdf.victor,tfIdf.school,tfIdf.future,tfIdf.atl,tfIdf.montgomery,tfIdf.farm,tfIdf.bdpa,tfIdf.laptops,tfIdf.programming,tfIdf.scary,tfIdf.bed,tfIdf.south,tfIdf.moves,tfIdf.literally,tfIdf.breathe,tfIdf.damage,tfIdf.reports,tfIdf.prompting,tfIdf.tremors,tfIdf.office,tfIdf.casualties,tfIdf.prayforjakarta,tfIdf.west,tfIdf.breaking,tfIdf.huge,tfIdf.rattled,tfIdf.residents,tfIdf.thankyou,tfIdf.inform,tfIdf.fine,tfIdf.release,tfIdf.loves,tfIdf.update,tfIdf.source,tfIdf.naturaldisasters,tfIdf.strongest,tfIdf.ve,tfIdf.yup,tfIdf.brush,tfIdf.ease,tfIdf.basics,tfIdf.august,tfIdf.expan,tfIdf.skills,tfIdf.write,tfIdf.updated,tfIdf.guide,tfIdf.apps,tfIdf.pretty,tfIdf.university,tfIdf.searching,tfIdf.reading,tfIdf.american,tfIdf.marrigge,tfIdf.girl,tfIdf.boy,tfIdf.arabian,tfIdf.books,tfIdf.pdf,tfIdf.teacher,tfIdf.budg,tfIdf.cambridge,tfIdf.diploma,tfIdf.internet,tfIdf.yall,tfIdf.boiz,tfIdf.ripcurrents,tfIdf.scared,tfIdf.gempa,tfIdf.mins,tfIdf.report,tfIdf.eqalerts,tfIdf.hello,tfIdf.isya,tfIdf.agency,tfIdf.geophysics,tfIdf.saturday,tfIdf.feared,tfIdf.disaster,tfIdf.guys,tfIdf.lifts,tfIdf.shallow,tfIdf.image,tfIdf.sw,tfIdf.epicenter,tfIdf.southern,tfIdf.southernsumatra,tfIdf.occurred,tfIdf.streets,tfIdf.fleeing,tfIdf.friday,tfIdf.evening,tfIdf.province,tfIdf.generating,tfIdf.indonesiaearthquake,tfIdf.risked,tfIdf.monitors,tfIdf.fecha,tfIdf.t

In [17]:
# Export
df.to_csv("temp/api_input.csv", index=False)


In [18]:
api_input = pd.read_csv("temp/api_input.csv")
api_input

,tweet_id,tweet_text,offset,sentiment,embeddings,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.mag,tfIdf.aeic,tfIdf.depth,tfIdf.utc,tfIdf.bmkg,tfIdf.location,tfIdf.earthquake,tfIdf.island,tfIdf.strikes,tfIdf.magnitude,tfIdf.sumatra,tfIdf.indonesia,tfIdf.news,tfIdf.including,tfIdf.main,tfIdf.centred,tfIdf.bali,tfIdf.java,tfIdf.powerful,tfIdf.islands,tfIdf.rocked,tfIdf.quake,tfIdf.safe,tfIdf.feel,tfIdf.jakarta,tfIdf.stay,tfIdf.app,tfIdf.southwest,tfIdf.amp,tfIdf.minutes,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.aug,tfIdf.fri,tfIdf.info,tfIdf.postgresql,tfIdf.united,tfIdf.alert,tfIdf.rate,tfIdf.scrum,tfIdf.developer,tfIdf.kingdom,tfIdf.anxiety,tfIdf.tsunami,tfIdf.bring,tfIdf.sign,tfIdf.rocks,tfIdf.warning,tfIdf.issued,tfIdf.sunda,tfIdf.lives,tfIdf.selat,tfIdf.banten,tfIdf.potential,tfIdf.sr,tfIdf.country,tfIdf.cities,tfIdf.live,tfIdf.hits,tfIdf.indonesias,tfIdf.dead,tfIdf.panic,tfIdf.indonesian,tfIdf.kills,tfIdf.strong,tfIdf.displaced,tfIdf.dwellers,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.struck,tfIdf.coastal,tfIdf.fear,tfIdf.super,tfIdf.drunk,tfIdf.spinning,tfIdf.wild,tfIdf.fears,tfIdf.ground,tfIdf.triggers,tfIdf.urged,tfIdf.capital,tfIdf.swaying,tfIdf.died,tfIdf.heart,tfIdf.authorities,tfIdf.hit,tfIdf.attack,tfIdf.buildings,tfIdf.damaged,tfIdf.houses,tfIdf.urging,tfIdf.rocking,tfIdf.officials,tfIdf.initially,tfIdf.coast,tfIdf.cancelled,tfIdf.populated,tfIdf.heavily,tfIdf.injured,tfIdf.worldnews,tfIdf.usgs,tfIdf.teacup,tfIdf.poodle,tfIdf.tiny,tfIdf.jual,tfIdf.black,tfIdf.betina,tfIdf.quality,tfIdf.toy,tfIdf.cbs,tfIdf.lifted,tfIdf.software,tfIdf.subscribe,tfIdf.learn,tfIdf.link,tfIdf.channel,tfIdf.pay,tfIdf.apwebier,tfIdf.click,tfIdf.languege,tfIdf.field,tfIdf.money,tfIdf.watch,tfIdf.success,tfIdf.earn,tfIdf.free,tfIdf.beautiful,tfIdf.destination,tfIdf.pray,tfIdf.assisting,tfIdf.chapter,tfIdf.code,tfIdf.languages,tfIdf.bright,tfIdf.students,tfIdf.competition,tfIdf.special,tfIdf.coding,tfIdf.victor,tfIdf.school,tfIdf.future,tfIdf.atl,tfIdf.montgomery,tfIdf.farm,tfIdf.bdpa,tfIdf.laptops,tfIdf.programming,tfIdf.scary,tfIdf.bed,tfIdf.south,tfIdf.moves,tfIdf.literally,tfIdf.breathe,tfIdf.damage,tfIdf.reports,tfIdf.prompting,tfIdf.tremors,tfIdf.office,tfIdf.casualties,tfIdf.prayforjakarta,tfIdf.west,tfIdf.breaking,tfIdf.huge,tfIdf.rattled,tfIdf.residents,tfIdf.thankyou,tfIdf.inform,tfIdf.fine,tfIdf.release,tfIdf.loves,tfIdf.update,tfIdf.source,tfIdf.naturaldisasters,tfIdf.strongest,tfIdf.ve,tfIdf.yup,tfIdf.brush,tfIdf.ease,tfIdf.basics,tfIdf.august,tfIdf.expan,tfIdf.skills,tfIdf.write,tfIdf.updated,tfIdf.guide,tfIdf.apps,tfIdf.pretty,tfIdf.university,tfIdf.searching,tfIdf.reading,tfIdf.american,tfIdf.marrigge,tfIdf.girl,tfIdf.boy,tfIdf.arabian,tfIdf.books,tfIdf.pdf,tfIdf.teacher,tfIdf.budg,tfIdf.cambridge,tfIdf.diploma,tfIdf.internet,tfIdf.yall,tfIdf.boiz,tfIdf.ripcurrents,tfIdf.scared,tfIdf.gempa,tfIdf.mins,tfIdf.report,tfIdf.eqalerts,tfIdf.hello,tfIdf.isya,tfIdf.agency,tfIdf.geophysics,tfIdf.saturday,tfIdf.feared,tfIdf.disaster,tfIdf.guys,tfIdf.lifts,tfIdf.shallow,tfIdf.image,tfIdf.sw,tfIdf.epicenter,tfIdf.southern,tfIdf.southernsumatra,tfIdf.occurred,tfIdf.streets,tfIdf.fleeing,tfIdf.friday,tfIdf.evening,tfIdf.province,tfIdf.generating,tfIdf.indonesiaearthquake,tfIdf.risked,tfIdf.monitors,tfIdf.fecha,tfIdf.t

In [19]:
# Drop the string categories
api_input.drop(['tweet_text'], axis = 1, inplace = True)
api_input.drop(['embeddings'], axis = 1, inplace = True)
api_input.drop(['sentiment'], axis = 1, inplace = True)

api_input.describe()

,tweet_id,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.mag,tfIdf.aeic,tfIdf.depth,tfIdf.utc,tfIdf.bmkg,tfIdf.location,tfIdf.earthquake,tfIdf.island,tfIdf.strikes,tfIdf.magnitude,tfIdf.sumatra,tfIdf.indonesia,tfIdf.news,tfIdf.including,tfIdf.main,tfIdf.centred,tfIdf.bali,tfIdf.java,tfIdf.powerful,tfIdf.islands,tfIdf.rocked,tfIdf.quake,tfIdf.safe,tfIdf.feel,tfIdf.jakarta,tfIdf.stay,tfIdf.app,tfIdf.southwest,tfIdf.amp,tfIdf.minutes,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.aug,tfIdf.fri,tfIdf.info,tfIdf.postgresql,tfIdf.united,tfIdf.alert,tfIdf.rate,tfIdf.scrum,tfIdf.developer,tfIdf.kingdom,tfIdf.anxiety,tfIdf.tsunami,tfIdf.bring,tfIdf.sign,tfIdf.rocks,tfIdf.warning,tfIdf.issued,tfIdf.sunda,tfIdf.lives,tfIdf.selat,tfIdf.banten,tfIdf.potential,tfIdf.sr,tfIdf.country,tfIdf.cities,tfIdf.live,tfIdf.hits,tfIdf.indonesias,tfIdf.dead,tfIdf.panic,tfIdf.indonesian,tfIdf.kills,tfIdf.strong,tfIdf.displaced,tfIdf.dwellers,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.struck,tfIdf.coastal,tfIdf.fear,tfIdf.super,tfIdf.drunk,tfIdf.spinning,tfIdf.wild,tfIdf.fears,tfIdf.ground,tfIdf.triggers,tfIdf.urged,tfIdf.capital,tfIdf.swaying,tfIdf.died,tfIdf.heart,tfIdf.authorities,tfIdf.hit,tfIdf.attack,tfIdf.buildings,tfIdf.damaged,tfIdf.houses,tfIdf.urging,tfIdf.rocking,tfIdf.officials,tfIdf.initially,tfIdf.coast,tfIdf.cancelled,tfIdf.populated,tfIdf.heavily,tfIdf.injured,tfIdf.worldnews,tfIdf.usgs,tfIdf.teacup,tfIdf.poodle,tfIdf.tiny,tfIdf.jual,tfIdf.black,tfIdf.betina,tfIdf.quality,tfIdf.toy,tfIdf.cbs,tfIdf.lifted,tfIdf.software,tfIdf.subscribe,tfIdf.learn,tfIdf.link,tfIdf.channel,tfIdf.pay,tfIdf.apwebier,tfIdf.click,tfIdf.languege,tfIdf.field,tfIdf.money,tfIdf.watch,tfIdf.success,tfIdf.earn,tfIdf.free,tfIdf.beautiful,tfIdf.destination,tfIdf.pray,tfIdf.assisting,tfIdf.chapter,tfIdf.code,tfIdf.languages,tfIdf.bright,tfIdf.students,tfIdf.competition,tfIdf.special,tfIdf.coding,tfIdf.victor,tfIdf.school,tfIdf.future,tfIdf.atl,tfIdf.montgomery,tfIdf.farm,tfIdf.bdpa,tfIdf.laptops,tfIdf.programming,tfIdf.scary,tfIdf.bed,tfIdf.south,tfIdf.moves,tfIdf.literally,tfIdf.breathe,tfIdf.damage,tfIdf.reports,tfIdf.prompting,tfIdf.tremors,tfIdf.office,tfIdf.casualties,tfIdf.prayforjakarta,tfIdf.west,tfIdf.breaking,tfIdf.huge,tfIdf.rattled,tfIdf.residents,tfIdf.thankyou,tfIdf.inform,tfIdf.fine,tfIdf.release,tfIdf.loves,tfIdf.update,tfIdf.source,tfIdf.naturaldisasters,tfIdf.strongest,tfIdf.ve,tfIdf.yup,tfIdf.brush,tfIdf.ease,tfIdf.basics,tfIdf.august,tfIdf.expan,tfIdf.skills,tfIdf.write,tfIdf.updated,tfIdf.guide,tfIdf.apps,tfIdf.pretty,tfIdf.university,tfIdf.searching,tfIdf.reading,tfIdf.american,tfIdf.marrigge,tfIdf.girl,tfIdf.boy,tfIdf.arabian,tfIdf.books,tfIdf.pdf,tfIdf.teacher,tfIdf.budg,tfIdf.cambridge,tfIdf.diploma,tfIdf.internet,tfIdf.yall,tfIdf.boiz,tfIdf.ripcurrents,tfIdf.scared,tfIdf.gempa,tfIdf.mins,tfIdf.report,tfIdf.eqalerts,tfIdf.hello,tfIdf.isya,tfIdf.agency,tfIdf.geophysics,tfIdf.saturday,tfIdf.feared,tfIdf.disaster,tfIdf.guys,tfIdf.lifts,tfIdf.shallow,tfIdf.image,tfIdf.sw,tfIdf.epicenter,tfIdf.southern,tfIdf.southernsumatra,tfIdf.occurred,tfIdf.streets,tfIdf.fleeing,tfIdf.friday,tfIdf.evening,tfIdf.province,tfIdf.generating,tfIdf.indonesiaearthquake,tfIdf.risked,tfIdf.monitors,tfIdf.fecha,tfIdf.temblor,tfIdf.sismo,tfIdf.prof,tf

In [20]:
# Group by tweet
feature_vector_input = api_input.groupby(['tweet_id']).agg('first')
feature_vector_input

,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.mag,tfIdf.aeic,tfIdf.depth,tfIdf.utc,tfIdf.bmkg,tfIdf.location,tfIdf.earthquake,tfIdf.island,tfIdf.strikes,tfIdf.magnitude,tfIdf.sumatra,tfIdf.indonesia,tfIdf.news,tfIdf.including,tfIdf.main,tfIdf.centred,tfIdf.bali,tfIdf.java,tfIdf.powerful,tfIdf.islands,tfIdf.rocked,tfIdf.quake,tfIdf.safe,tfIdf.feel,tfIdf.jakarta,tfIdf.stay,tfIdf.app,tfIdf.southwest,tfIdf.amp,tfIdf.minutes,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.aug,tfIdf.fri,tfIdf.info,tfIdf.postgresql,tfIdf.united,tfIdf.alert,tfIdf.rate,tfIdf.scrum,tfIdf.developer,tfIdf.kingdom,tfIdf.anxiety,tfIdf.tsunami,tfIdf.bring,tfIdf.sign,tfIdf.rocks,tfIdf.warning,tfIdf.issued,tfIdf.sunda,tfIdf.lives,tfIdf.selat,tfIdf.banten,tfIdf.potential,tfIdf.sr,tfIdf.country,tfIdf.cities,tfIdf.live,tfIdf.hits,tfIdf.indonesias,tfIdf.dead,tfIdf.panic,tfIdf.indonesian,tfIdf.kills,tfIdf.strong,tfIdf.displaced,tfIdf.dwellers,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.struck,tfIdf.coastal,tfIdf.fear,tfIdf.super,tfIdf.drunk,tfIdf.spinning,tfIdf.wild,tfIdf.fears,tfIdf.ground,tfIdf.triggers,tfIdf.urged,tfIdf.capital,tfIdf.swaying,tfIdf.died,tfIdf.heart,tfIdf.authorities,tfIdf.hit,tfIdf.attack,tfIdf.buildings,tfIdf.damaged,tfIdf.houses,tfIdf.urging,tfIdf.rocking,tfIdf.officials,tfIdf.initially,tfIdf.coast,tfIdf.cancelled,tfIdf.populated,tfIdf.heavily,tfIdf.injured,tfIdf.worldnews,tfIdf.usgs,tfIdf.teacup,tfIdf.poodle,tfIdf.tiny,tfIdf.jual,tfIdf.black,tfIdf.betina,tfIdf.quality,tfIdf.toy,tfIdf.cbs,tfIdf.lifted,tfIdf.software,tfIdf.subscribe,tfIdf.learn,tfIdf.link,tfIdf.channel,tfIdf.pay,tfIdf.apwebier,tfIdf.click,tfIdf.languege,tfIdf.field,tfIdf.money,tfIdf.watch,tfIdf.success,tfIdf.earn,tfIdf.free,tfIdf.beautiful,tfIdf.destination,tfIdf.pray,tfIdf.assisting,tfIdf.chapter,tfIdf.code,tfIdf.languages,tfIdf.bright,tfIdf.students,tfIdf.competition,tfIdf.special,tfIdf.coding,tfIdf.victor,tfIdf.school,tfIdf.future,tfIdf.atl,tfIdf.montgomery,tfIdf.farm,tfIdf.bdpa,tfIdf.laptops,tfIdf.programming,tfIdf.scary,tfIdf.bed,tfIdf.south,tfIdf.moves,tfIdf.literally,tfIdf.breathe,tfIdf.damage,tfIdf.reports,tfIdf.prompting,tfIdf.tremors,tfIdf.office,tfIdf.casualties,tfIdf.prayforjakarta,tfIdf.west,tfIdf.breaking,tfIdf.huge,tfIdf.rattled,tfIdf.residents,tfIdf.thankyou,tfIdf.inform,tfIdf.fine,tfIdf.release,tfIdf.loves,tfIdf.update,tfIdf.source,tfIdf.naturaldisasters,tfIdf.strongest,tfIdf.ve,tfIdf.yup,tfIdf.brush,tfIdf.ease,tfIdf.basics,tfIdf.august,tfIdf.expan,tfIdf.skills,tfIdf.write,tfIdf.updated,tfIdf.guide,tfIdf.apps,tfIdf.pretty,tfIdf.university,tfIdf.searching,tfIdf.reading,tfIdf.american,tfIdf.marrigge,tfIdf.girl,tfIdf.boy,tfIdf.arabian,tfIdf.books,tfIdf.pdf,tfIdf.teacher,tfIdf.budg,tfIdf.cambridge,tfIdf.diploma,tfIdf.internet,tfIdf.yall,tfIdf.boiz,tfIdf.ripcurrents,tfIdf.scared,tfIdf.gempa,tfIdf.mins,tfIdf.report,tfIdf.eqalerts,tfIdf.hello,tfIdf.isya,tfIdf.agency,tfIdf.geophysics,tfIdf.saturday,tfIdf.feared,tfIdf.disaster,tfIdf.guys,tfIdf.lifts,tfIdf.shallow,tfIdf.image,tfIdf.sw,tfIdf.epicenter,tfIdf.southern,tfIdf.southernsumatra,tfIdf.occurred,tfIdf.streets,tfIdf.fleeing,tfIdf.friday,tfIdf.evening,tfIdf.province,tfIdf.generating,tfIdf.indonesiaearthquake,tfIdf.risked,tfIdf.monitors,tfIdf.fecha,tfIdf.temblor,tfIdf.sismo,tfIdf.prof,tfIdf.gmt,t

## Load the labelled data

These are generated in 0_Labels.ipynb

In [21]:
# dtypes needs to be specified or long ints will change
labels_df = pd.read_json("../../../data/input/raw/data/2020/2020-A/labels/TRECIS-2018-2020A-labels.json", dtype={} )

# replace the event with a numeric value
labels_df = labels_df.replace({'eventID': event_int_map})

# Count the number of labels
labels_df['num'] = labels_df['postCategories'].str.len()


# Map the priority to a numeric value
labels_df = labels_df.replace({"postPriority": priority_mapping})

# Split categories and map to numeric values
category_list = pd.DataFrame(labels_df["postCategories"].to_list(), columns=['cat1', 'cat2', 'cat3',
                                                                   'cat4', 'cat5', 'cat6',
                                                                   'cat7', 'cat8', 'cat9', 'cat10'])


# Map the categories to numeric values
category_list = category_list.applymap(lambda s: mymap.get(s) if s in mymap else s)


# Join back onto our original list
labels = labels_df.join(category_list)

# Drop the string categories
labels.drop(['postCategories'], axis = 1, inplace = True)

# Tidy
labels = labels.drop(['eventName', 'eventDescription', 'eventType'], axis=1)


# Fill the NaN slots with 0
labels = labels.fillna("0")

# Export
labels.to_csv("../labels.csv", index=False)
labels = clean_dataset_int(labels)
#labels = clean_dataset_new(labels)

labels

,eventID,postID,postPriority,num,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10
0,16.0,7.267628e+16,2.5,3.0,7.0,10.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16.0,7.267840e+16,10.0,4.0,22.0,18.0,10.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0
2,16.0,7.268240e+16,2.5,2.0,21.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16.0,7.269393e+16,5.0,3.0,23.0,10.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16.0,7.269856e+16,2.5,3.0,23.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42946,47.0,1.162000e+18,2.5,1.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42947,47.0,1.162005e+18,2.5,3.0,13.0,15.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42948,47.0,1.162005e+18,2.5,2.0,13.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42949,47.0,1.162006e+18,2.5,3.0,13.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Train.csv


Merges the input feature vector with the labels

In [22]:
train = pd.merge(labels, feature_vector_input, left_on = 'postID', right_on = 'tweet_id', how = 'inner')

train = train.reset_index()
train.fillna(0, inplace=True)

train.replace([np.inf, -np.inf], np.nan, inplace=True)

train = train.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

train.to_csv("../train.csv", index=False)

train

,index,eventID,postID,postPriority,num,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.mag,tfIdf.aeic,tfIdf.depth,tfIdf.utc,tfIdf.bmkg,tfIdf.location,tfIdf.earthquake,tfIdf.island,tfIdf.strikes,tfIdf.magnitude,tfIdf.sumatra,tfIdf.indonesia,tfIdf.news,tfIdf.including,tfIdf.main,tfIdf.centred,tfIdf.bali,tfIdf.java,tfIdf.powerful,tfIdf.islands,tfIdf.rocked,tfIdf.quake,tfIdf.safe,tfIdf.feel,tfIdf.jakarta,tfIdf.stay,tfIdf.app,tfIdf.southwest,tfIdf.amp,tfIdf.minutes,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.aug,tfIdf.fri,tfIdf.info,tfIdf.postgresql,tfIdf.united,tfIdf.alert,tfIdf.rate,tfIdf.scrum,tfIdf.developer,tfIdf.kingdom,tfIdf.anxiety,tfIdf.tsunami,tfIdf.bring,tfIdf.sign,tfIdf.rocks,tfIdf.warning,tfIdf.issued,tfIdf.sunda,tfIdf.lives,tfIdf.selat,tfIdf.banten,tfIdf.potential,tfIdf.sr,tfIdf.country,tfIdf.cities,tfIdf.live,tfIdf.hits,tfIdf.indonesias,tfIdf.dead,tfIdf.panic,tfIdf.indonesian,tfIdf.kills,tfIdf.strong,tfIdf.displaced,tfIdf.dwellers,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.struck,tfIdf.coastal,tfIdf.fear,tfIdf.super,tfIdf.drunk,tfIdf.spinning,tfIdf.wild,tfIdf.fears,tfIdf.ground,tfIdf.triggers,tfIdf.urged,tfIdf.capital,tfIdf.swaying,tfIdf.died,tfIdf.heart,tfIdf.authorities,tfIdf.hit,tfIdf.attack,tfIdf.buildings,tfIdf.damaged,tfIdf.houses,tfIdf.urging,tfIdf.rocking,tfIdf.officials,tfIdf.initially,tfIdf.coast,tfIdf.cancelled,tfIdf.populated,tfIdf.heavily,tfIdf.injured,tfIdf.worldnews,tfIdf.usgs,tfIdf.teacup,tfIdf.poodle,tfIdf.tiny,tfIdf.jual,tfIdf.black,tfIdf.betina,tfIdf.quality,tfIdf.toy,tfIdf.cbs,tfIdf.lifted,tfIdf.software,tfIdf.subscribe,tfIdf.learn,tfIdf.link,tfIdf.channel,tfIdf.pay,tfIdf.apwebier,tfIdf.click,tfIdf.languege,tfIdf.field,tfIdf.money,tfIdf.watch,tfIdf.success,tfIdf.earn,tfIdf.free,tfIdf.beautiful,tfIdf.destination,tfIdf.pray,tfIdf.assisting,tfIdf.chapter,tfIdf.code,tfIdf.languages,tfIdf.bright,tfIdf.students,tfIdf.competition,tfIdf.special,tfIdf.coding,tfIdf.victor,tfIdf.school,tfIdf.future,tfIdf.atl,tfIdf.montgomery,tfIdf.farm,tfIdf.bdpa,tfIdf.laptops,tfIdf.programming,tfIdf.scary,tfIdf.bed,tfIdf.south,tfIdf.moves,tfIdf.literally,tfIdf.breathe,tfIdf.damage,tfIdf.reports,tfIdf.prompting,tfIdf.tremors,tfIdf.office,tfIdf.casualties,tfIdf.prayforjakarta,tfIdf.west,tfIdf.breaking,tfIdf.huge,tfIdf.rattled,tfIdf.residents,tfIdf.thankyou,tfIdf.inform,tfIdf.fine,tfIdf.release,tfIdf.loves,tfIdf.update,tfIdf.source,tfIdf.naturaldisasters,tfIdf.strongest,tfIdf.ve,tfIdf.yup,tfIdf.brush,tfIdf.ease,tfIdf.basics,tfIdf.august,tfIdf.expan,tfIdf.skills,tfIdf.write,tfIdf.updated,tfIdf.guide,tfIdf.apps,tfIdf.pretty,tfIdf.university,tfIdf.searching,tfIdf.reading,tfIdf.american,tfIdf.marrigge,tfIdf.girl,tfIdf.boy,tfIdf.arabian,tfIdf.books,tfIdf.pdf,tfIdf.teacher,tfIdf.budg,tfIdf.cambridge,tfIdf.diploma,tfIdf.internet,...,tfIdf.isya,tfIdf.agency,tfIdf.geophysics,tfIdf.saturday,tfIdf.feared,tfIdf.disaster,tfIdf.guys,tfIdf.lifts,tfIdf.shallow,tfIdf.image,tfIdf.sw,tfIdf.epicenter,tfIdf.southern,tfIdf.southernsumatra,tfIdf.occurred,tfIdf.streets,tfIdf.fleeing,tfIdf.friday,tfIdf.evening,tfIdf.province,tfIdf.generating,tfIdf.indonesiaearthquake,tfIdf.risked,tfIdf.monitors,tfIdf.fecha,tfIdf.temblor,tfIdf.sismo,tfIdf.prof,tfIdf.gmt,tfIdf.latitude,tfIdf.tra

# Test.csv


Drops the categories, number of categories and priority so we can make our prediction

In [23]:
test = train

# Drop cat*
test.drop(list(test.filter(regex = 'cat\d+$')), axis = 1, inplace = True)

# Drop priority / num (of labels)
test.drop(['postPriority', 'num'], axis = 1, inplace = True)

# export
test.to_csv("../test.csv", index=False)

test

,index,eventID,postID,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.mag,tfIdf.aeic,tfIdf.depth,tfIdf.utc,tfIdf.bmkg,tfIdf.location,tfIdf.earthquake,tfIdf.island,tfIdf.strikes,tfIdf.magnitude,tfIdf.sumatra,tfIdf.indonesia,tfIdf.news,tfIdf.including,tfIdf.main,tfIdf.centred,tfIdf.bali,tfIdf.java,tfIdf.powerful,tfIdf.islands,tfIdf.rocked,tfIdf.quake,tfIdf.safe,tfIdf.feel,tfIdf.jakarta,tfIdf.stay,tfIdf.app,tfIdf.southwest,tfIdf.amp,tfIdf.minutes,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.aug,tfIdf.fri,tfIdf.info,tfIdf.postgresql,tfIdf.united,tfIdf.alert,tfIdf.rate,tfIdf.scrum,tfIdf.developer,tfIdf.kingdom,tfIdf.anxiety,tfIdf.tsunami,tfIdf.bring,tfIdf.sign,tfIdf.rocks,tfIdf.warning,tfIdf.issued,tfIdf.sunda,tfIdf.lives,tfIdf.selat,tfIdf.banten,tfIdf.potential,tfIdf.sr,tfIdf.country,tfIdf.cities,tfIdf.live,tfIdf.hits,tfIdf.indonesias,tfIdf.dead,tfIdf.panic,tfIdf.indonesian,tfIdf.kills,tfIdf.strong,tfIdf.displaced,tfIdf.dwellers,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.struck,tfIdf.coastal,tfIdf.fear,tfIdf.super,tfIdf.drunk,tfIdf.spinning,tfIdf.wild,tfIdf.fears,tfIdf.ground,tfIdf.triggers,tfIdf.urged,tfIdf.capital,tfIdf.swaying,tfIdf.died,tfIdf.heart,tfIdf.authorities,tfIdf.hit,tfIdf.attack,tfIdf.buildings,tfIdf.damaged,tfIdf.houses,tfIdf.urging,tfIdf.rocking,tfIdf.officials,tfIdf.initially,tfIdf.coast,tfIdf.cancelled,tfIdf.populated,tfIdf.heavily,tfIdf.injured,tfIdf.worldnews,tfIdf.usgs,tfIdf.teacup,tfIdf.poodle,tfIdf.tiny,tfIdf.jual,tfIdf.black,tfIdf.betina,tfIdf.quality,tfIdf.toy,tfIdf.cbs,tfIdf.lifted,tfIdf.software,tfIdf.subscribe,tfIdf.learn,tfIdf.link,tfIdf.channel,tfIdf.pay,tfIdf.apwebier,tfIdf.click,tfIdf.languege,tfIdf.field,tfIdf.money,tfIdf.watch,tfIdf.success,tfIdf.earn,tfIdf.free,tfIdf.beautiful,tfIdf.destination,tfIdf.pray,tfIdf.assisting,tfIdf.chapter,tfIdf.code,tfIdf.languages,tfIdf.bright,tfIdf.students,tfIdf.competition,tfIdf.special,tfIdf.coding,tfIdf.victor,tfIdf.school,tfIdf.future,tfIdf.atl,tfIdf.montgomery,tfIdf.farm,tfIdf.bdpa,tfIdf.laptops,tfIdf.programming,tfIdf.scary,tfIdf.bed,tfIdf.south,tfIdf.moves,tfIdf.literally,tfIdf.breathe,tfIdf.damage,tfIdf.reports,tfIdf.prompting,tfIdf.tremors,tfIdf.office,tfIdf.casualties,tfIdf.prayforjakarta,tfIdf.west,tfIdf.breaking,tfIdf.huge,tfIdf.rattled,tfIdf.residents,tfIdf.thankyou,tfIdf.inform,tfIdf.fine,tfIdf.release,tfIdf.loves,tfIdf.update,tfIdf.source,tfIdf.naturaldisasters,tfIdf.strongest,tfIdf.ve,tfIdf.yup,tfIdf.brush,tfIdf.ease,tfIdf.basics,tfIdf.august,tfIdf.expan,tfIdf.skills,tfIdf.write,tfIdf.updated,tfIdf.guide,tfIdf.apps,tfIdf.pretty,tfIdf.university,tfIdf.searching,tfIdf.reading,tfIdf.american,tfIdf.marrigge,tfIdf.girl,tfIdf.boy,tfIdf.arabian,tfIdf.books,tfIdf.pdf,tfIdf.teacher,tfIdf.budg,tfIdf.cambridge,tfIdf.diploma,tfIdf.internet,tfIdf.yall,tfIdf.boiz,tfIdf.ripcurrents,tfIdf.scared,tfIdf.gempa,tfIdf.mins,tfIdf.report,tfIdf.eqalerts,tfIdf.hello,tfIdf.isya,tfIdf.agency,tfIdf.geophysics,tfIdf.saturday,tfIdf.feared,tfIdf.disaster,tfIdf.guys,tfIdf.lifts,tfIdf.shallow,tfIdf.image,tfIdf.sw,tfIdf.epicenter,tfIdf.southern,tfIdf.southernsumatra,tfIdf.occurred,tfIdf.streets,tfIdf.fleeing,tfIdf.friday,tfIdf.evening,tfIdf.province,tfIdf.generating,tfIdf.indonesiaearthquake,tfIdf.risked,tfIdf.monitors,tfIdf.fecha,tfIdf.temblor,tfIdf.sismo,t

# Igel

This notebook implements `Igel`

> Igel supports all sklearn's machine learning functionality,

Igel's supported models:

        +--------------------+----------------------------+-------------------------+
        |      regression    |        classification      |        clustering       |
        +--------------------+----------------------------+-------------------------+
        |   LinearRegression |         LogisticRegression |                  KMeans |
        |              Lasso |                      Ridge |     AffinityPropagation |
        |          LassoLars |               DecisionTree |                   Birch |
        | BayesianRegression |                  ExtraTree | AgglomerativeClustering |
        |    HuberRegression |               RandomForest |    FeatureAgglomeration |
        |              Ridge |                 ExtraTrees |                  DBSCAN |
        |  PoissonRegression |                        SVM |         MiniBatchKMeans |
        |      ARDRegression |                  LinearSVM |    SpectralBiclustering |
        |  TweedieRegression |                      NuSVM |    SpectralCoclustering |
        | TheilSenRegression |            NearestNeighbor |      SpectralClustering |
        |    GammaRegression |              NeuralNetwork |               MeanShift |
        |   RANSACRegression | PassiveAgressiveClassifier |                  OPTICS |
        |       DecisionTree |                 Perceptron |                    ---- |
        |          ExtraTree |               BernoulliRBM |                    ---- |
        |       RandomForest |           BoltzmannMachine |                    ---- |
        |         ExtraTrees |       CalibratedClassifier |                    ---- |
        |                SVM |                   Adaboost |                    ---- |
        |          LinearSVM |                    Bagging |                    ---- |
        |              NuSVM |           GradientBoosting |                    ---- |
        |    NearestNeighbor |        BernoulliNaiveBayes |                    ---- |
        |      NeuralNetwork |      CategoricalNaiveBayes |                    ---- |
        |         ElasticNet |       ComplementNaiveBayes |                    ---- |
        |       BernoulliRBM |         GaussianNaiveBayes |                    ---- |
        |   BoltzmannMachine |      MultinomialNaiveBayes |                    ---- |
        |           Adaboost |                       ---- |                    ---- |
        |            Bagging |                       ---- |                    ---- |
        |   GradientBoosting |                       ---- |                    ---- |
        +--------------------+----------------------------+-------------------------+

In [24]:
params = {
        'cmd': 'fit',    
        'data_path': "../train.csv",
        'yaml_path': 'yaml/multi.yaml'  # DecisionTree
}

Igel(**params)

INFO - Entered CLI args: {'cmd': 'fit', 'data_path': '../train.csv', 'yaml_path': 'yaml/multi.yaml'}
INFO - Executing command: fit ...
INFO - reading data from ../train.csv
INFO - You passed the configurations as a yaml file.
INFO - your chosen configuration: {'dataset': {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default', 'scale': {'method': 'standard', 'target': 'inputs'}}}, 'model': {'type': 'regression', 'algorithm': 'DecisionTree'}, 'target': ['postPriority', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'num']}
INFO - dataset_props: {'split': {'test_size': 0.2, 'shuffle': True, 'stratify': 'default', 'scale': {'method': 'standard', 'target': 'inputs'}}} 
model_props: {'type': 'regression', 'algorithm': 'DecisionTree'} 
 target: ['postPriority', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'num'] 

INFO - dataset shape: (105, 509)
INFO - dataset attributes: ['index', 'eventID', 'postID', 'postPr

INFO - y shape: (105, 12) and x shape: (105, 497)
INFO - Solving a regression problem using ===> DecisionTree
INFO - model arguments: 
None
INFO - executing a DecisionTreeRegressor algorithm...
INFO - predicting multiple targets detected. Hence, the model will be automatically converted to a multioutput model
INFO - cross validation is not provided
INFO - Folder /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results already exists
WARNING - data in the /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results folder will be overridden. If you don't want this, then move the current /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results to another path
INFO - Successfully created the directory in /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results 
INFO - model saved successfully and can be found in the /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results folder
INFO - split option detected. The performance will be a

In [25]:
params = {
        'cmd': 'predict',    
        'data_path': "../test.csv",
        'yaml_path': 'yaml/hyper.yaml'
}
Igel(**params)

INFO - Entered CLI args: {'cmd': 'predict', 'data_path': '../test.csv', 'yaml_path': 'yaml/hyper.yaml'}
INFO - Executing command: predict ...
INFO - reading data from ../test.csv
INFO - path of the pre-fitted model => /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results/model.sav
INFO - loading from /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results/model.sav
INFO - dataset shape: (105, 497)
INFO - dataset attributes: ['index', 'eventID', 'postID', 'offset', 'features.numb_of_urls', 'features.numb_of_hashtags', 'features.numb_of_personal_pronouns', 'features.numb_of_present_tenses', 'features.weighted_length', 'features.permillage', 'features.tweet_created_at', 'features.tweet_id_str', 'features.positive_sentiment', 'features.negative_sentiment', 'features.numb_of_mentions', 'features.numb_of_media', 'features.numb_of_past_tenses', 'features.numb_of_weird_chars', 'features.numb_of_questions', 'features.numb_of_emoticons', 'features.numb_of_swearing_wor

INFO - predictions shape: (105, 12) | shape len: 2
INFO - predict on targets: ['postPriority', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'num']
INFO - saving the predictions to /Users/mark/Documents/Calamity/src/jupyter/1-notebooks/model_results/predictions.csv


# Predictions

1. View the raw predictions
2. Map the labels to their High Level Information Types
3. Merge the predictions back into the training set


In [26]:

def is_neg_predictions(predictions):
    predictions = predictions.sort_values(by=['postPriority'])
    predictions = predictions[(predictions > 0).all(1)]
    predictions.round()

    
predictions = pd.read_csv("model_results/predictions.csv")
predictions

#is_neg_predictions(predictions)

,postPriority,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,num
0,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2.5,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
100,2.5,8.0,25.0,13.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,5.0
101,2.5,23.0,25.0,13.0,15.0,7.0,17.0,0.0,0.0,0.0,0.0,6.0
102,2.5,23.0,25.0,13.0,7.0,17.0,0.0,0.0,0.0,0.0,0.0,5.0
103,2.5,23.0,25.0,13.0,6.0,15.0,17.0,0.0,0.0,0.0,0.0,6.0


### Merge the new predictions back onto dataframe with the missing columns

In [27]:
# Map the labels to their High Level Information Types
cat_list = predictions.filter(regex='cat', axis=1).round().applymap(lambda x: highCategoriser[x])

#
predictions = cat_list.combine_first(predictions)

# Merge the predictions back into the training set
df = test.merge(predictions, left_index=True, right_index=True)


# Append the predicted categories to a list in a new column
df['predicted_categories'] = df[['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10']].values.tolist()


# Get the number of categories into something we can use to index
df['num'] = df['num'].astype(float).astype(int)

# Remove categories beyond what the tweet is predicted to have
df['categories'] = df.apply(lambda x: x['predicted_categories'][0:x['num']], axis=1)


df

,index,eventID,postID,offset,features.numb_of_urls,features.numb_of_hashtags,features.numb_of_personal_pronouns,features.numb_of_present_tenses,features.weighted_length,features.permillage,features.tweet_created_at,features.tweet_id_str,features.positive_sentiment,features.negative_sentiment,features.numb_of_mentions,features.numb_of_media,features.numb_of_past_tenses,features.numb_of_weird_chars,features.numb_of_questions,features.numb_of_emoticons,features.numb_of_swearing_word,features.numb_of_slang_words,features.numb_of_intensifiers,features.tweet_length,features.userFollowersCount,features.userFriendsCount,features.user_numb_of_tweets,features.user_list_count,features.dict_precision,features.dict_recall,features.dict_f_measure,features.offset,features.is_verified,tfIdf.mag,tfIdf.aeic,tfIdf.depth,tfIdf.utc,tfIdf.bmkg,tfIdf.location,tfIdf.earthquake,tfIdf.island,tfIdf.strikes,tfIdf.magnitude,tfIdf.sumatra,tfIdf.indonesia,tfIdf.news,tfIdf.including,tfIdf.main,tfIdf.centred,tfIdf.bali,tfIdf.java,tfIdf.powerful,tfIdf.islands,tfIdf.rocked,tfIdf.quake,tfIdf.safe,tfIdf.feel,tfIdf.jakarta,tfIdf.stay,tfIdf.app,tfIdf.southwest,tfIdf.amp,tfIdf.minutes,tfIdf.erthquakes,tfIdf.volcanoes,tfIdf.ago,tfIdf.aug,tfIdf.fri,tfIdf.info,tfIdf.postgresql,tfIdf.united,tfIdf.alert,tfIdf.rate,tfIdf.scrum,tfIdf.developer,tfIdf.kingdom,tfIdf.anxiety,tfIdf.tsunami,tfIdf.bring,tfIdf.sign,tfIdf.rocks,tfIdf.warning,tfIdf.issued,tfIdf.sunda,tfIdf.lives,tfIdf.selat,tfIdf.banten,tfIdf.potential,tfIdf.sr,tfIdf.country,tfIdf.cities,tfIdf.live,tfIdf.hits,tfIdf.indonesias,tfIdf.dead,tfIdf.panic,tfIdf.indonesian,tfIdf.kills,tfIdf.strong,tfIdf.displaced,tfIdf.dwellers,tfIdf.killed,tfIdf.reuters,tfIdf.person,tfIdf.undersea,tfIdf.struck,tfIdf.coastal,tfIdf.fear,tfIdf.super,tfIdf.drunk,tfIdf.spinning,tfIdf.wild,tfIdf.fears,tfIdf.ground,tfIdf.triggers,tfIdf.urged,tfIdf.capital,tfIdf.swaying,tfIdf.died,tfIdf.heart,tfIdf.authorities,tfIdf.hit,tfIdf.attack,tfIdf.buildings,tfIdf.damaged,tfIdf.houses,tfIdf.urging,tfIdf.rocking,tfIdf.officials,tfIdf.initially,tfIdf.coast,tfIdf.cancelled,tfIdf.populated,tfIdf.heavily,tfIdf.injured,tfIdf.worldnews,tfIdf.usgs,tfIdf.teacup,tfIdf.poodle,tfIdf.tiny,tfIdf.jual,tfIdf.black,tfIdf.betina,tfIdf.quality,tfIdf.toy,tfIdf.cbs,tfIdf.lifted,tfIdf.software,tfIdf.subscribe,tfIdf.learn,tfIdf.link,tfIdf.channel,tfIdf.pay,tfIdf.apwebier,tfIdf.click,tfIdf.languege,tfIdf.field,tfIdf.money,tfIdf.watch,tfIdf.success,tfIdf.earn,tfIdf.free,tfIdf.beautiful,tfIdf.destination,tfIdf.pray,tfIdf.assisting,tfIdf.chapter,tfIdf.code,tfIdf.languages,tfIdf.bright,tfIdf.students,tfIdf.competition,tfIdf.special,tfIdf.coding,tfIdf.victor,tfIdf.school,tfIdf.future,tfIdf.atl,tfIdf.montgomery,tfIdf.farm,tfIdf.bdpa,tfIdf.laptops,tfIdf.programming,tfIdf.scary,tfIdf.bed,tfIdf.south,tfIdf.moves,tfIdf.literally,tfIdf.breathe,tfIdf.damage,tfIdf.reports,tfIdf.prompting,tfIdf.tremors,tfIdf.office,tfIdf.casualties,tfIdf.prayforjakarta,tfIdf.west,tfIdf.breaking,tfIdf.huge,tfIdf.rattled,tfIdf.residents,tfIdf.thankyou,tfIdf.inform,tfIdf.fine,tfIdf.release,tfIdf.loves,tfIdf.update,tfIdf.source,tfIdf.naturaldisasters,tfIdf.strongest,tfIdf.ve,tfIdf.yup,tfIdf.brush,tfIdf.ease,tfIdf.basics,tfIdf.august,tfIdf.expan,tfIdf.skills,tfIdf.write,tfIdf.updated,tfIdf.guide,tfIdf.apps,tfIdf.pretty,tfIdf.university,tfIdf.searching,tfIdf.reading,tfIdf.american,tfIdf.marrigge,tfIdf.girl,tfIdf.boy,tfIdf.arabian,tfIdf.books,tfIdf.pdf,tfIdf.teacher,tfIdf.budg,tfIdf.cambridge,tfIdf.diploma,tfIdf.internet,tfIdf.yall,tfIdf.boiz,tfIdf.ripcurrents,tfIdf.scared,tfIdf.gempa,tfIdf.mins,tfIdf.report,tfIdf.eqalerts,tfIdf.hello,tfIdf.isya,tfIdf.agency,tfIdf.geophysics,...,tfIdf.occurred,tfIdf.streets,tfIdf.fleeing,tfIdf.friday,tfIdf.evening,tfIdf.province,tfIdf.generating,tfIdf.indonesiaearthquake,tfIdf.risked,tfIdf.monitors,tfIdf.fecha,tfIdf.temblor,tfIdf.sismo,tfIdf.prof,tfIdf.gmt,tfIdf.latitude,tfIdf.tracker,tfIdf.estimated,tfIdf.earthquakes,tfIdf.petajoule,tfIdf.energy,tfIdf.longitude,tfIdf.bengkulu,tfIdf.l

## Export

Export in the TRECIS format

In [28]:
# write to .run file
with open("marks2.run" , "w") as out_file:
    for row in df.drop_duplicates(subset="postID").itertuples():
        #print("row:", row)
        content = [
            "TRECIS-CTIT-H-Test-0" + str(int(row.eventID)),
            "Q0",          
            np.int64(row.postID),   
            getattr(row, 'Index'),  #ToDo: Fix?
            #row.priority,
            str(priority_scorer[str(round(row.postPriority))[:3]]),  #ToDo: Fix
            row.categories,
            "marksrun2"
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")